In [ ]:
from __future__ import absolute_import, division, print_function
import numpy as np
import collections
import math
import os
import random
from six.moves import urllib
import io
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output, Image, display, HTML
import tensorflow as tf
import tensorflow_hub as hub
import sklearn.metrics as sk_metrics
import time
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers, layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout, Flatten
from tensorflow.keras import backend as K
from keras.utils.vis_utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report
from keras.applications import xception

In [ ]:
train_path = '../input/datasetfinal/dataset70-10-20/train/Fruit'
validation_path = '../input/datasetfinal/dataset70-10-20/val/Fruit'
test_path = '../input/datasetfinal/dataset70-10-20/test/Fruit'

In [ ]:
fruits_classes=[]
for root, subdirectories, files in os.walk(train_path):
    for subdirectory in subdirectories:
        fruits_classes.append(subdirectory)
print(len(fruits_classes))

In [ ]:
train_batches  = ImageDataGenerator().flow_from_directory(
    train_path, target_size=(224, 224), classes = fruits_classes, batch_size = 50)

validation_batches  = ImageDataGenerator().flow_from_directory(
    validation_path, target_size=(224, 224), classes = fruits_classes, batch_size = 10)

test_batches  = ImageDataGenerator().flow_from_directory(
    test_path, target_size=(224, 224), classes = fruits_classes, batch_size = 10)

## Data Augmentation

In [ ]:
train_datagen_aug = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest',
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator_aug = train_datagen_aug.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=25,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=(224, 224),
        batch_size=1,
        shuffle = False,
        class_mode='categorical')

val_generator = test_datagen.flow_from_directory(
        validation_path,
        target_size=(224, 224),
        batch_size=10,
        class_mode='categorical')

In [ ]:
train_filenames = train_generator_aug.filenames
steps_train = len(train_filenames)/train_generator_aug.batch_size

val_filenames = val_generator.filenames
steps_val = len(val_filenames)/val_generator.batch_size

filenames_test = test_generator.filenames
nb_samples_test= len(filenames_test)
true_classes = test_generator.classes
target_names = fruits_classes

# Xception

In [ ]:
model = xception.Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3),classes=28,pooling='avg')
model.summary()

In [ ]:
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
count = 0
for layer in model.layers:
    count += 1
count

In [ ]:
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[-10:]:
    layer.trainable = True

In [ ]:
x= model.output
x= tf.keras.layers.Dropout(.4)(x)
output=Dense(28,activation='softmax')(x)
fruits_xception= Model(model.input,output)

In [ ]:
fruits_xception.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(1e-4),
              metrics=['acc'])

es = EarlyStopping(monitor='val_accuracy', min_delta=0.01,patience= 3, verbose=1)
mc = ModelCheckpoint("./fruits_best_xception.h5", monitor='val_accuracy', verbose=1, save_best_only=True)
cb=[es, mc]

fit_generator = fruits_xception.fit(train_generator_aug,
                                    steps_per_epoch=steps_train,
                                    epochs=50,
                                    validation_data=val_generator,
                                    validation_steps=steps_val,
                                    verbose=1,
                                    callbacks=cb)

In [ ]:
acc = fit_generator.history['acc']
val_acc = fit_generator.history['val_acc']
loss = fit_generator.history['loss']
val_loss = fit_generator.history['val_loss']

epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
fruits_xception.save('fruits_xception.h5')

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(fruits_xception)
tflite_model = converter.convert()

with open('fruits_xception.tflite', 'wb') as f:
    f.write(tflite_model)

## Predict

In [ ]:
fruits_pred= fruits_xception.predict(test_generator, steps = nb_samples_test, verbose=2)
fruits_pred= fruits_pred.argmax(axis=1)

### Classification Report

In [ ]:
print(classification_report(true_classes, fruits_pred, target_names=target_names))